In [ ]:
import os, sys

import cv2
import numpy as np
from tqdm import tqdm

from data_preprocess import DataSetLoader


In [ ]:

def merge_images(paths):
    print(paths[0])
    b, g, r = cv2.split(cv2.imread(paths[0]))  # ambient_occlusion
    _, _, r = cv2.split(cv2.imread(paths[4]))  # specular
    _, _, b = cv2.split(cv2.imread(paths[5]))  # translucency
    
    merged_image = cv2.merge((b, g, r))
    merged_image = cv2.cvtColor(merged_image, cv2.COLOR_BGR2RGB)

    # 1, 2, 3 인덱스의 이미지를 읽음
    images = [cv2.imread(paths[i]) for i in [1, 2, 3]]
    images = [cv2.cvtColor(i, cv2.COLOR_BGR2RGB) for i in images]
    
    # 합쳐진 이미지를 마지막에 추가
    images.append(merged_image)
    
    result = np.stack(images, axis=0)
    result = np.transpose(result, (0,3,1,2))  # HWC에서 CHW로 변환

    return result

In [ ]:
a = DataSetLoader()
a.get_all_file_paths("./Data")
print(a.image_paths[0][0])
os.path.isfile(a.image_paths[0][0][0])

In [ ]:
dataset_dir = './Data'
out = './data_prep_4chan'
if not os.path.exists(out):
    os.makedirs(out)

In [ ]:
dataset_dir = './Data'
out = './data_prep_4chan'
if not os.path.exists(out):
    os.makedirs(out)

for folder_name in tqdm(os.listdir(dataset_dir)):
    folder_path = os.path.join(dataset_dir, folder_name)
    for sub in os.listdir(folder_path):
        sub_folder_path = os.path.join(folder_path, sub)
        if os.path.isdir(sub_folder_path):
            paths = [
                os.path.join(sub_folder_path, 'ambient_occlusion.png'),
                os.path.join(sub_folder_path, 'diffuse.png'),
                os.path.join(sub_folder_path, 'light_normalized.png'),
                os.path.join(sub_folder_path, 'normal.png'),
                os.path.join(sub_folder_path, 'specular.png'),
                os.path.join(sub_folder_path, 'translucency.png')
            ]
            merged_images = merge_images(paths)
            np.save(os.path.join(out, f'data_{sub}.npy'), merged_images)